In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from datetime import datetime

In [2]:
df = pd.read_excel(r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\displacements\unhcr-prmn-displacement-dataset-3.xlsx")
len(df)

57666

In [3]:
path = "./som_adm_ocha_itos_20230308_shp/" 
states_gdf = gpd.read_file(path+"som_admbnda_adm1_ocha_20230308.shp") #sdn_admbnda_adm1_cbs_nic_ssa_20200831.shp")

In [4]:
df.head()

,Month End,Year Week,Current (Arrival) Region,Current (Arrival) District,Previous (Departure) Region,Previous (Departure) District,Reason,Current (Arrival) Priority Need,Number of Individuals
0,30/04/2016,201614,Awdal,Borama,Awdal,Borama,Drought related,Food,6
1,30/04/2016,201617,Awdal,Borama,Awdal,Borama,Drought related,Livelihood Support,7
2,30/04/2016,201616,Awdal,Borama,Awdal,Borama,Flood,Food,5
3,30/04/2016,201614,Awdal,Borama,Awdal,Lughaye,Drought related,Food,8
4,31/03/2016,201611,Awdal,Borama,Awdal,Lughaye,Drought related,Food,9


In [5]:
df_d = df[df['Reason']=='Drought related']

In [6]:
v= df_d["Month End"]
v=v.values
dt = [datetime.strptime(v[i], "%d/%m/%Y") for i in range(len(v))]
q=[]

for i in range(len(dt)):
    q.append(datetime.timestamp(dt[i]))
    
df_d.insert(loc=3, column='date_timestamp', value=q)
df_d = df_d.sort_values("date_timestamp")

df_d['Month End'] = pd.to_datetime(df_d['Month End'], dayfirst=True)
#df_d=df_d.set_index('Month End')

In [7]:
df_d.head()

,Month End,Year Week,Current (Arrival) Region,date_timestamp,Current (Arrival) District,Previous (Departure) Region,Previous (Departure) District,Reason,Current (Arrival) Priority Need,Number of Individuals
2698,2016-01-31,201604,Woqooyi Galbeed,1.454195e+09,Hargeysa,Middle Shabelle,Cadale,Drought related,Food,6
2129,2016-01-31,201603,Mudug,1.454195e+09,Gaalkacyo,Mudug,Hobyo,Drought related,Humanitarian Aid,187
2125,2016-01-31,201604,Mudug,1.454195e+09,Gaalkacyo,Mudug,Hobyo,Drought related,Health,765
2124,2016-01-31,201603,Mudug,1.454195e+09,Gaalkacyo,Mudug,Hobyo,Drought related,Health,240
2123,2016-01-31,201601,Mudug,1.454195e+09,Gaalkacyo,Mudug,Hobyo,Drought related,Health,216


In [8]:
len(df_d)

37537

In [ ]:
# Aggregate the datetime objects by month
aggregated_data = df_d.groupby([pd.Grouper(key='Month End', freq='M'),'Current (Arrival) Region'])['Number of Individuals'].sum()

In [ ]:
aggregated_data=aggregated_data.to_frame().reset_index()

In [ ]:
regions_nev = states_gdf[states_gdf.ADM1_EN.isin(states_gdf["ADM1_EN"])]

In [17]:
#in df remove all rows that contain a NaN
df = df.dropna()
df.reset_index(drop=True, inplace=True)
len(df)

2736